In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"XGBoost version: {xgb.__version__}")

Forecast Sales Revenue

In [ ]:
# Change input features here
business_date = '01/07/2025'
venue_xref_id = '6ba501a035db17bd76f4379d22146d404a92643f527c961c2e954bf652846385'
concept = 'FAMILY_DINING'
city = 'Seattle'
country = 'US'

In [ ]:
forecast = pd.DataFrame(data={'business_date': [pd.Timestamp(business_date)],
                                'venue_xref_id': [venue_xref_id],
                                'concept': [concept],
                                'city': [city],
                                'country': [country]})

In [ ]:
forecast

In [ ]:
def create_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create new features for the data.
    """
    df = df.copy()

    # Add new features
    df['day_of_week'] = df['business_date'].dt.day_of_week
    df['day_of_year'] = df['business_date'].dt.day_of_year
    df['month'] = df['business_date'].dt.month
    df['quarter'] = df['business_date'].dt.quarter

    return df

def encode_categorical(df: pd.DataFrame, encoder, features: list) -> pd.DataFrame:
    df = df.copy()

    # Encode categorical data
    df = encoder.transform(df)

    # Specify categorical data type for XGBoost
    df[features] = df[features].astype('category')

    return df

In [ ]:
# Load the encoder to transform the forecast data
with open('encoder.obj', 'rb') as f:
    encoder = pickle.load(f)

In [ ]:
FEATURES = ['venue_xref_id', 'concept', 'city', 'country', 
            'day_of_week', 'day_of_year', 'month', 'quarter']

forecast = create_features(forecast)
forecast.drop(columns='business_date', inplace=True)

forecast = encode_categorical(forecast, encoder, FEATURES)
forecast

In [ ]:
touchbistro_forecaster = xgb.XGBRegressor()
touchbistro_forecaster.load_model('CxC_TouchBistro_Forecaster.json')

In [ ]:
sales_revenue_with_tax = touchbistro_forecaster.predict(forecast)
print(f'Forecasted Sales Revenue for {venue_xref_id}:')
print(f'{business_date}: ${sales_revenue_with_tax.item():.2f}')